In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
print(sys.path)
FOLDERNAME = 'finalproject'
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\Drive/$FOLDERNAME

print(sys.path)
%load_ext autoreload
%autoreload 2

Mounted at /content/drive
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']
/content/drive/MyDrive/finalproject
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/My Drive/finalproject']


In [ ]:
!pip install transformers
!pip install positional_encodings
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import BertModel
import json
from torch.utils.data import Dataset, DataLoader
import os
import torch
import torch.nn as nn
from torch import Tensor
import math
from torch.nn import functional as F
import copy
from typing import Optional, List

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch.optim as optim

In [ ]:
import numpy as np
import pandas as pd
import os 
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
from our_detr import build


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# **Loading dataset, Preprocessing**

# **Create dataset for target aspect prediction**

In [ ]:
import csv
targets= pd.read_csv("targets.csv")
list = []

for v in targets['sentiment']:
  dicts = {}
  x = v.split(',')
  z = torch.tensor([]).cuda()
  for i in range(len(x)):
    y = torch.tensor([float(x[i])]).cuda()
    z = torch.cat((z,y),0).cuda()
  dicts['sentiment'] = z.cuda()
  list.append(dicts)

for l in range(len(targets['sentence'])):
  x = targets['sentence'][l].split(',')
  z =''
  for i in range(len(x)):
   z = z + x[i]
  list[l]['sentence'] = z
for l in range(len(targets['target_enc'])):
  x = targets['target_enc'][l].split(',')
  z = torch.tensor([]).cuda()
  for i in range(len(x)):
    y = torch.tensor([int(x[i])]).cuda()
    z = torch.cat((z,y),0).cuda() 
  list[l]['target'] = z.long()

for l in range(len(targets['aspect_enc'])):
  x = targets['aspect_enc'][l].split(',')
  z = torch.tensor([]).cuda()
  for i in range(len(x)):
    y = torch.tensor([int(x[i])]).cuda()
    z = torch.cat((z,y),0).cuda() 
  list[l]['aspect'] = z.long()

In [ ]:
print(targets['aspect_enc'].unique())

['0' '1' '0,0' '1,1' '2' '3' '2,2,2' '1,2' '0,0,0' '3,0' '1,0,1' '1,0']


# Get Bert and put preprocessed data in




In [ ]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


# INPUT TEXT -> WORD EMBEDDING 


In [ ]:
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []
# get sentences from the preprocessed data.
sentences = targets["sentence"]
# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 32,           # Pad & truncate all sentences.
                        padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences)
print('Token IDs:', input_ids)

Original:  0      Royal Mail chairman Donald Brydon set to step ...
1      Stakes High for AstraZeneca Heart Drug Facing ...
2      UPDATE 1-Dairy Crest loses a third of Morrison...
3      Insight hires Aviva's David Hillier for multi-...
4      Primark racks up a happy Christmas after stron...
                             ...                        
431    Aviva, M&G suspend property funds as investors...
432    UK housing market steadies after Brexit dip, P...
433    BRIEF-Aviva aims to increase dividend pay-out ...
434       Builder Persimmon hails 6% rise in house sales
435    EasyJet attracts more passengers in June but s...
Name: sentence, Length: 436, dtype: object
Token IDs: tensor([[  101,  2548,  5653,  ...,     0,     0,     0],
        [  101,  7533,  2152,  ...,     0,     0,     0],
        [  101, 10651,  1015,  ...,     0,     0,     0],
        ...,
        [  101,  4766,  1011,  ...,     0,     0,     0],
        [  101, 12508,  2566,  ...,     0,     0,     0],
     

torch.Size([32])


# **Get positional encoding to put into decoder**

# ***Making FULL transformer model with Bert & decoder ***

# **Initialize Transformer**

# **Initialize DETR with my model**

# **DETR 만들기**

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'

model, criterion = build(436) 
y = input_ids.cuda()
z = attention_masks.cuda()
model.to(device)
criterion.to(device)
n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('number of params:', n_parameters)

optimizer = torch.optim.Adam(params=model.parameters(),
                 lr=1e-4,
                 weight_decay=1e-4,
                 eps=1e-8)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 200)
               


number of params: 48653801


In [ ]:
epoch_loss = 0
model.train()
criterion.train()

PATH = "model.pt"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
criterion.load_state_dict(checkpoint['criterion_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
lr_scheduler.load_state_dict(checkpoint['lr_state_dict'])
for epoch in range(0,20000): 
  x = model(y,z)
  loss_dict = criterion(x, list)
  weight_dict = criterion.weight_dict
  loss = criterion(x, list)
  losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
  optimizer.zero_grad()
  losses.backward()
  torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
  optimizer.step()
  epoch_loss += losses.item()
  lr_scheduler.step()
  print(losses)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/content/drive/My Drive/finalproject/our_detr.py:154: UserWarning: Using a target size (torch.Size([498])) that is different to the input size (torch.Size([498, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss_sentiment = F.mse_loss(src_sentiment, target_sentiment, reduction='mean')


tensor(6.2080, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.3206, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2736, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2505, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2669, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2513, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2777, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.3483, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2665, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2376, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2482, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2753, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2706, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2401, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2282, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.2346, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.1966, device='cuda:0', grad_fn=<AddBack

KeyboardInterrupt: ignored

In [ ]:
PATH = "model.pt"
torch.save({
            'model_state_dict': model.state_dict(),
            'criterion_state_dict': criterion.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss,
            'lr_state_dict':lr_scheduler.state_dict()
            }, PATH)

In [ ]:
model, _ = build(1) 
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])



<All keys matched successfully>

In [ ]:
y = input_ids.cuda()
z = attention_masks.cuda()
model.to(device)
criterion.to(device)

SetCriterion(
  (matcher): HungarianMatcher()
)

In [ ]:
out = model(input_ids[1].unsqueeze(0),attention_masks[1].unsqueeze(0))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [ ]:
out_logit, out_logit2, out_sentiment = out['pred_aspect'], out['pred_target'], out['pred_sentiment']
prob1 = F.softmax(out_logit, -1)
prob2 = F.softmax(out_logit2, -1)
scores2, labels2 = prob2[...,:-1].max(-1)
scores, labels = prob1[...,:-1].max(-1)

In [ ]:
print(torch.argmax(scores))

NameError: ignored

In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()
print(output)

tensor(2.1164, grad_fn=<NllLossBackward0>)


In [ ]:
print(input)

tensor([[ 1.0774,  1.2802, -0.1061,  1.5755,  1.9523],
        [ 1.7794, -0.4060, -0.5727,  1.4729,  0.4263],
        [ 1.1334, -0.8765, -0.3775,  0.3331,  1.1246]], requires_grad=True)


In [ ]:
target

tensor([3, 4, 2])